# 개요
* GPT활용교육을 들은 내용을 별도로 정리
  * 1일차 : 네이버API 활용 : GPT를 활용한 변수 분석 등
  * 2일차 : beautifulsoup활용한 뉴스 및 증권사리포트 크롤링
  * 3일차 : 뉴스 및 증권사리포트 데이터 전처리 
  * 4일차 : 감성점수 변환
  * 5일차 : 백테스팅 실습

# 1일차 실습내용 정리

## 실습 : pykrx로 코스피 Ticker와 종목이름 가져오기

In [ ]:
from pykrx import stock

# 코스피 상장 종목 코드 리스트 가져오기
tickers = stock.get_market_ticker_list(market="KOSPI")

In [ ]:
import pandas as pd

df_tickers = pd.DataFrame(tickers, columns=['Ticker'])
df_tickers['Name'] = df_tickers['Ticker'].apply(lambda x:stock.get_market_ticker_name(x))
df_tickers.head(10)

,Ticker,Name
0,095570,AJ네트웍스
1,006840,AK홀딩스
2,027410,BGF
3,282330,BGF리테일
4,138930,BNK금융지주
5,001460,BYC
6,001465,BYC우
7,001040,CJ
8,079160,CJ CGV
9,00104K,CJ4우(전환)


## 실습 : 네이버 API로 뉴스 검색해보기

* API키는 본문이 아닌 keys.json 파일로 별도로 저장 후 .gitignore를 활용

In [ ]:
import requests
import json

# 네이버 API 인증 정보
with open("keys.json", "r") as f:
    data = json.load(f)

client_id = data.get("naver_client_id")
client_secret = data.get("naver_client_secret")

In [ ]:
# 기초 실습

## 검색 URL
keyword = '삼성전자' # 검색 키워드
data_type = 'json'  # 반환 형식(xml 또는 json)
display = 10 # 표시할 검색결과 수 (~100, 기본값 10)
start = 1    # 검색 시작 위치(~100, 기본값 1)
sort = 'sim' # 정렬방식 (모두 내림차순. sim 정확도순 / date 날짜순 )
url = f"https://openapi.naver.com/v1/search/news.{data_type}?query={keyword}&display={display}&start={start}&sort={sort}"

## API 요청 헤더
headers = {'X-Naver-Client-Id':client_id,
           'X-Naver-Client-Secret':client_secret}

## API호출 테스트 (200 : 정상수신)
response = requests.get(url, headers=headers)
response

<Response [200]>

In [ ]:
pd.set_option('display.max_colwidth', 20)

In [ ]:
## 결과값 가져오기

if response.status_code == 200: # 정상 수신된 경우
    news_data = response.json() # 결과값을 json형태로 파싱 (items 안에 {}형태로 각 기사가 있음)
    news_items = news_data.get('items', []) # items의 []안의 데이터를 가져옴
    df = pd.DataFrame(news_items)
    
    from IPython.display import display
    display(df)
else:
    print(f"ErrorCode : {response.status_code}")

,title,originallink,link,description,pubDate
0,"SK하이닉스, 1분기 D램 시...",https://www.news...,https://n.news.n...,SK하이닉스가 올해 1분기 <...,"Tue, 03 Jun 2025..."
1,<b>삼성전자</b>도 제쳤다...,https://www.hank...,https://n.news.n...,SK하이닉스가 올해 1분기 <...,"Tue, 03 Jun 2025..."
2,"SK하이닉스, 1분기 글로벌 ...",https://www.news...,https://n.news.n...,SK하이닉스(000660)가 ...,"Tue, 03 Jun 2025..."
3,"대만 TSMC 출신 임원, <...",https://biz.chos...,https://n.news.n...,<b>삼성전자</b>가 파운드...,"Tue, 03 Jun 2025..."
4,삼성페이 3시간30분 ‘먹통’...,https://www.hani...,https://n.news.n...,<b>삼성전자</b>의 모바일...,"Mon, 02 Jun 2025..."
5,"SK하이닉스, <b>삼성전자<...",https://www.imae...,https://n.news.n...,SK하이닉스가 인공지능(AI)...,"Tue, 03 Jun 2025..."
6,"<b>삼성전자</b>, 구글 ...",https://www.mk.c...,https://n.news.n...,<b>삼성전자</b>가 인공지...,"Mon, 02 Jun 2025..."
7,<b>삼성</b>페이 결제 오...,https://www.yna....,https://n.news.n...,오전 7시께부터 오류 시작 추...,"Mon, 02 Jun 2025..."
8,[단독] <b>삼성전자</b>...,http://www.fnnew...,https://n.news.n...,<b>삼성전자</b> 반도체부...,"Mon, 02 Jun 2025..."
9,"<b>삼성전자</b>, 4일 ...",https://zdnet.co...,https://n.news.n...,<b>삼성전자</b> 파운드리...,"Mon, 02 Jun 2025..."


In [ ]:
## 데이터 형태 확인(news_data) : items 안에 {}형태로 각 기사가 있음
news_data

```json
{'lastBuildDate': 'Tue, 03 Jun 2025 21:43:30 +0900',
 'total': 3931030,
 'start': 1,
 'display': 10,
 'items': [{'title': 'SK하이닉스, 1분기 D램 시장 1위…&quot;<b>삼성전자</b>도 제쳤다&quot;',
   'originallink': 'https://www.newsis.com/view/NISX20250603_0003200190',
   'link': 'https://n.news.naver.com/mnews/article/003/0013283316?sid=101',
   'description': 'SK하이닉스가 올해 1분기 <b>삼성전자</b>를 제치고 글로벌 D램 시장 1위에 올랐다. 3일 대만 시장조사업체 트렌드포스에 따르면 SK하이닉스는 올 1분기 매출 97억1800만달러를 기록해, 36% 점유율로 1위를 기록했다.... ',
   'pubDate': 'Tue, 03 Jun 2025 17:45:00 +0900'},
...
  {'title': '<b>삼성전자</b>, 4일 SAFE 포럼 개최...내실 다지기',
   'originallink': 'https://zdnet.co.kr/view/?no=20250602154629',
   'link': 'https://n.news.naver.com/mnews/article/092/0002376758?sid=105',
   'description': '<b>삼성전자</b> 파운드리(반도체 위탁생산)가 내실 다지기에 들어갔다. 회사는 매년 삼성 파운드리 포럼과 SAFE(Samsung Advanced Foundry Ecosystem) 포럼을 동시 개최해왔으나, 올해는 SAFE포럼만 개최한다. 아울러 행사 국가, 시간... ',
   'pubDate': 'Mon, 02 Jun 2025 16:12:00 +0900'}
   ]
   }
  ```

In [ ]:
## 데이터 형태 확인(news_items) : (앞의 lastBuildDate 등을 지우고)items의 []안의 데이터를 가져옴
news_items

```json
[{'title': 'SK하이닉스, 1분기 D램 시장 1위…&quot;<b>삼성전자</b>도 제쳤다&quot;',
  'originallink': 'https://www.newsis.com/view/NISX20250603_0003200190',
  'link': 'https://n.news.naver.com/mnews/article/003/0013283316?sid=101',
  'description': 'SK하이닉스가 올해 1분기 <b>삼성전자</b>를 제치고 글로벌 D램 시장 1위에 올랐다. 3일 대만 시장조사업체 트렌드포스에 따르면 SK하이닉스는 올 1분기 매출 97억1800만달러를 기록해, 36% 점유율로 1위를 기록했다.... ',
  'pubDate': 'Tue, 03 Jun 2025 17:45:00 +0900'},
...
 {'title': '<b>삼성전자</b>, 4일 SAFE 포럼 개최...내실 다지기',
  'originallink': 'https://zdnet.co.kr/view/?no=20250602154629',
  'link': 'https://n.news.naver.com/mnews/article/092/0002376758?sid=105',
  'description': '<b>삼성전자</b> 파운드리(반도체 위탁생산)가 내실 다지기에 들어갔다. 회사는 매년 삼성 파운드리 포럼과 SAFE(Samsung Advanced Foundry Ecosystem) 포럼을 동시 개최해왔으나, 올해는 SAFE포럼만 개최한다. 아울러 행사 국가, 시간... ',
  'pubDate': 'Mon, 02 Jun 2025 16:12:00 +0900'}
  ]
```

## 실습 : 각 종목에 대해 네이버 API로 뉴스 검색(이전 실습 활용)

In [ ]:
import requests
import json

# 네이버 API 인증 정보
with open("keys.json", "r") as f:
    data = json.load(f)

client_id = data.get("naver_client_id")
client_secret = data.get("naver_client_secret")

In [ ]:
# 전체 데이터 보관할 빈 데이터 프레임
main_df = pd.DataFrame()

# 검색할 대상
list_kwd = df_tickers['Name'].tolist()

for each_kwd in list_kwd[:10]: # 962개는 너무 많으므로 10개만 검색해보기 
    ## 검색 URL
    data_type = 'json'  # 반환 형식(xml 또는 json)
    display = 10 # 표시할 검색결과 수 (~100, 기본값 10)
    start = 1    # 검색 시작 위치(~100, 기본값 1)
    sort = 'sim' # 정렬방식 (모두 내림차순. sim 정확도순 / date 날짜순 )
    url = f"https://openapi.naver.com/v1/search/news.{data_type}?query={each_kwd}&display={display}&start={start}&sort={sort}"

    ## API 요청 헤더
    headers = {'X-Naver-Client-Id':client_id,
            'X-Naver-Client-Secret':client_secret}

    ## API호출
    response = requests.get(url, headers=headers)
    
    ## 결과값 처리
    if response.status_code == 200: # 정상 수신된 경우
        ### json형태로 데이터 처리
        news_data = response.json() # 결과값을 json형태로 파싱 (items 안에 {}형태로 각 기사가 있음)
        news_items = news_data.get('items', []) # items의 []안의 데이터를 가져옴
        ### 데이터프레임으로 변환 & 구분용 키워드 부여
        df = pd.DataFrame(news_items)
        df['키워드명'] = each_kwd
        ### 데이터프레임 병합(타 키워드와 병합)
        main_df = pd.concat([main_df, df])
    else:
        print(f"{each_kwd}에 대한 ErrorCode : {response.status_code}")

In [ ]:
# 결과 확인
main_df[['키워드명', 'title', 'originallink', 'link', 'description', 'pubDate']]

,키워드명,title,originallink,link,description,pubDate
0,AJ네트웍스,"<b>AJ네트웍스</b>, 서...",https://www.news...,https://n.news.n...,<b>AJ네트웍스</b>는 서...,"Mon, 19 May 2025..."
1,AJ네트웍스,"<b>AJ네트웍스</b>, 서...",http://www.edail...,https://n.news.n...,<b>AJ네트웍스</b>(09...,"Mon, 19 May 2025..."
2,AJ네트웍스,"﻿<b>AJ네트웍스</b>, ...",https://daily.ha...,https://daily.ha...,사진=<b>AJ네트웍스</b>...,"Mon, 19 May 2025..."
3,AJ네트웍스,"<b>AJ네트웍스</b>, 서...",https://www.klne...,https://www.klne...,<b>AJ네트웍스</b>㈜가 ...,"Mon, 19 May 2025..."
4,AJ네트웍스,"<b>AJ네트웍스</b>, '...",https://www.hell...,https://www.hell...,<b>AJ네트웍스</b>㈜가 ...,"Mon, 19 May 2025..."
...,...,...,...,...,...,...
5,CJ4우(전환),"<b>CJ4우(전환)</b>,...",http://news.wowt...,https://n.news.n...,코스피시장에서 <b>CJ4우(...,"Tue, 10 Dec 2019..."
6,CJ4우(전환),[복합기업 업종 11시 현재 ...,https://www.wide...,https://www.wide...,복합기업 업종 11시 상위 종...,"Wed, 18 Sep 2019..."
7,CJ4우(전환),[종목시세] <b>CJ4우(전...,http://www.finte...,http://www.finte...,FINTECHPOST <b>C...,"Thu, 12 Sep 2019..."
8,CJ4우(전환),[코스피 주가 하락률 Top ...,http://www.finte...,http://www.finte...,FINTECHPOST <b>C...,"Fri, 09 Aug 2019..."


## 처음 제공된 기초 코드 샘플

In [ ]:
# 처음 제공된 기초 코드
from tqdm import tqdm
import pandas as pd
import urllib.request
import json

naver_API_df = pd.DataFrame(columns=['company', 'title', 'link', 'description', 'pubDate'])

for kospi in tqdm(kospi_name):
    # 검색할 키워드 설정 (UTF-8 인코딩)
    encText = urllib.parse.quote(kospi)

    # 뉴스 검색 API URL 설정 (날짜순 정렬, 10개의 기사 요청)
    url = "https://openapi.naver.com/v1/search/news?query=" + encText + "&display=10&sort=sim"  # 정확도순, 10개의 기사

    # API 요청 설정
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)

    # API 호출 및 응답 처리
    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if rescode == 200:
        response_body = response.read().decode('utf-8')

        # JSON 데이터를 파싱
        news_data = json.loads(response_body)

        # 각 뉴스 아이템에서 필요한 정보를 추출하고 출력
        for item in news_data['items']:
            title = item['title'].replace('<b>', '').replace('</b>', '')  # HTML 태그 제거
            link = item['link']
            description = item['description'].replace('<b>', '').replace('</b>', '')
            pubDate = item['pubDate']

            # 저장
            naver_API_df.loc[len(naver_API_df)] = [kospi, title, link, description, pubDate]

    else:
        print("Error Code:" + str(rescode))

naver_API_df.to_csv('뉴스API.csv', index=False, encoding='utf-8-sig')